##  Pratice example of NLP applied to sentimental analysis

Dataset Downloaded from [kaggle](https://www.kaggle.com/columbine/imdb-dataset-sentiment-analysis-in-csv-format/data). We have only focus on the train and test dataset.

In [364]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [365]:
# Don't forget to change the directory 
X= pd.read_csv('./Train.csv', header = 0)
Y= pd.read_csv('./Test.csv', header = 0)
##This below dataset combine all data of the train and test table. 
#That will be useful for generating the vocabulary
data_text = pd.read_csv ('./data_text.csv',  header =0)

In [366]:
print(len(X))
print(len(Y))
print(len(data_text))

40000
5000
45000


In [367]:
X_train = X.iloc[:, 0]
Y_train = X.iloc[:, 1]

In [368]:
X_test = Y.iloc[:, 0]
Y_test = Y.iloc[:, 1]

In [369]:
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [349]:
#data_text = df.iloc[0].astype(str)

In [376]:
data_text = np.array(data_text)
data_text = df.iloc[0].astype(str)

## Pre-processing raw data

As mentioned in the section ([Understand RNN](https://github.com/Sohou08/NLP-applied-to-Sentiment-Analysis/blob/master/Understand%20RNN.md
)), the network can not work direclty on test-strings. The first step is to convert word to interger. The process is called in our flowchart 'Tokenizer'. By choice, we are construct the tokenizer on  10000 most popular words from the data-set.

In [377]:
num_words = 10000

In [378]:
tokenizer = Tokenizer(num_words=num_words)

In [379]:
%%time
tokenizer.fit_on_texts(data_text)

Wall time: 3 ms


In [380]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [381]:
tokenizer.word_index

{'the': 1,
 'a': 2,
 'of': 3,
 'i': 4,
 'and': 5,
 'thunderbirds': 6,
 'school': 7,
 'to': 8,
 'be': 9,
 'was': 10,
 'his': 11,
 'all': 12,
 'my': 13,
 'we': 14,
 'wanted': 15,
 'one': 16,
 'with': 17,
 'sapiens': 18,
 'grew': 19,
 'up': 20,
 'b': 21,
 '1965': 22,
 'watching': 23,
 'loving': 24,
 'mates': 25,
 'at': 26,
 'watched': 27,
 'played': 28,
 'before': 29,
 'during': 30,
 'lunch': 31,
 'after': 32,
 'virgil': 33,
 'or': 34,
 'scott': 35,
 'no': 36,
 'alan': 37,
 'counting': 38,
 'down': 39,
 'from': 40,
 '5': 41,
 'became': 42,
 'an': 43,
 'art': 44,
 'form': 45,
 'took': 46,
 'children': 47,
 'see': 48,
 'movie': 49,
 'hoping': 50,
 'they': 51,
 'would': 52,
 'get': 53,
 'glimpse': 54,
 'what': 55,
 'loved': 56,
 'as': 57,
 'child': 58,
 'how': 59,
 'bitterly': 60,
 'disappointing': 61,
 'only': 62,
 'high': 63,
 'point': 64,
 'snappy': 65,
 'theme': 66,
 'tune': 67,
 'not': 68,
 'that': 69,
 'it': 70,
 'could': 71,
 'compare': 72,
 'original': 73,
 'score': 74,
 'thankfully'

The tokenizer will be used to convert all texts in the training and test set to lists of these tokens.

In [382]:
X_train_tokens = tokenizer.texts_to_sequences(X_train)

In [383]:
X_train[1]

"When I put this movie in my DVD player, and sat down with a coke and some chips, I had some expectations. I was hoping that this movie would contain some of the strong-points of the first movie: Awsome animation, good flowing story, excellent voice cast, funny comedy and a kick-ass soundtrack. But, to my disappointment, not any of this is to be found in Atlantis: Milo's Return. Had I read some reviews first, I might not have been so let down. The following paragraph will be directed to those who have seen the first movie, and who enjoyed it primarily for the points mentioned.<br /><br />When the first scene appears, your in for a shock if you just picked Atlantis: Milo's Return from the display-case at your local videoshop (or whatever), and had the expectations I had. The music feels as a bad imitation of the first movie, and the voice cast has been replaced by a not so fitting one. (With the exception of a few characters, like the voice of Sweet). The actual drawings isnt that bad, 

In [384]:
np.array(X_train_tokens[1])

array([  4,  49,  93,  13,   5,  39,  17,   2,   5,   4,   4,  10,  50,
        69,  49,  52,   3,   1,   3,   1,  49,   5,   2, 107,   8,  13,
        68,   3,   8,   9,  93,   4,   4,  68,  39,   1,   9,   8,   1,
        49,   5,  70,   1,   1,  93,   2,  40,   1,  26,  34,   5,   1,
         4,   1,  57,   2,   3,   1,  49,   5,   1,   2,  68,  16,  17,
         1,   3,   2,   1,   3,   1,  69, 107,   1,  93,   2,   1,  57,
         3,   1,  14,   1, 107,  68,   4,   2,   4,  34,   8,   1,   1,
        49,  34,   2,   2,   3,  49,  57,   2,  49,  71,  81,  49,   8,
         5,   4,  70,   9,   2,  49, 107,   4,  70,   2,  84,   2,  80,
        34])

In [385]:
X_test_tokens = tokenizer.texts_to_sequences(X_test)

## Padding and Truncating Data


The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. There are two ways to do that: either ensure that all sequences in the entire data-set have the same length, or write a custom data-generator that ensures the sequences have the same length within each batch.
The first solution seems more simpler. But if we use the length of the longest sequence in the data-set, then we are wasting a lot of memory. This is particularly important for larger data-sets.
So in order to make a compromise, we will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.
First we count the number of tokens in all the sequences in the data-set.

In [386]:
num_tokens = [len(tokens) for tokens in X_train_tokens + X_test_tokens]
num_tokens = np.array(num_tokens)

In [388]:
# average number of tokens in a sequence 
np.mean(num_tokens)


78.78204444444444

In [390]:
# maximum number of tokens in a sequence
np.max(num_tokens)

803

In [391]:
#The max number of tokens we will allow is set to the average plus 2 standard deviations.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

193

In [392]:
#This covers about 95% of the data-set.
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9455333333333333


When padding or truncating the sequences that have a different length, we need to determine if we want to do this padding or truncating 'pre' or 'post'. If a sequence is truncated, it means that a part of the sequence is simply thrown away. If a sequence is padded, it means that zeros are added to the sequence.
So the choice of 'pre' or 'post' can be important because it determines whether we throw away the first or last part of a sequence when truncating, and it determines whether we add zeros to the beginning or end of the sequence when padding. This may confuse the Recurrent Neural Network.

In [393]:
pad = 'pre'

In [394]:
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [396]:
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [399]:
#We have now transformed the training-set into one big matrix of integers (tokens) with this shape:
X_train_pad.shape

(40000, 193)

In [400]:
X_test_pad.shape

(5000, 193)

In [401]:
#For example, we had the following sequence of tokens above:
np.array(X_train_tokens[1])

array([  4,  49,  93,  13,   5,  39,  17,   2,   5,   4,   4,  10,  50,
        69,  49,  52,   3,   1,   3,   1,  49,   5,   2, 107,   8,  13,
        68,   3,   8,   9,  93,   4,   4,  68,  39,   1,   9,   8,   1,
        49,   5,  70,   1,   1,  93,   2,  40,   1,  26,  34,   5,   1,
         4,   1,  57,   2,   3,   1,  49,   5,   1,   2,  68,  16,  17,
         1,   3,   2,   1,   3,   1,  69, 107,   1,  93,   2,   1,  57,
         3,   1,  14,   1, 107,  68,   4,   2,   4,  34,   8,   1,   1,
        49,  34,   2,   2,   3,  49,  57,   2,  49,  71,  81,  49,   8,
         5,   4,  70,   9,   2,  49, 107,   4,  70,   2,  84,   2,  80,
        34])

## Tokenizer Inverse Map
Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. We have to do by ourself.



In [403]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

#Helper-function for converting a list of tokens back to a string of words.

def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

X_train[1]

"When I put this movie in my DVD player, and sat down with a coke and some chips, I had some expectations. I was hoping that this movie would contain some of the strong-points of the first movie: Awsome animation, good flowing story, excellent voice cast, funny comedy and a kick-ass soundtrack. But, to my disappointment, not any of this is to be found in Atlantis: Milo's Return. Had I read some reviews first, I might not have been so let down. The following paragraph will be directed to those who have seen the first movie, and who enjoyed it primarily for the points mentioned.<br /><br />When the first scene appears, your in for a shock if you just picked Atlantis: Milo's Return from the display-case at your local videoshop (or whatever), and had the expectations I had. The music feels as a bad imitation of the first movie, and the voice cast has been replaced by a not so fitting one. (With the exception of a few characters, like the voice of Sweet). The actual drawings isnt that bad, 

In [404]:
tokens_to_string(X_train_tokens[1])

'i movie in my and down with a and i i was hoping that movie would of the of the movie and a but to my not of to be in i i not down the be to the movie and it the the in a from the at or and the i the as a of the movie and the a not one with the of a the of the that but the in a the as of the we the but not i a i or to the the movie or a a of movie as a movie could still movie to and i it be a movie but i it a series a channel or'

# Create the Recurrent Neural Network


In [405]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

model.add(GRU(units=16, return_sequences=True))

model.add(GRU(units=8, return_sequences=True))

model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 193, 8)            80000     
_________________________________________________________________
gru_15 (GRU)                 (None, 193, 16)           1248      
_________________________________________________________________
gru_16 (GRU)                 (None, 193, 8)            624       
_________________________________________________________________
gru_17 (GRU)                 (None, 4)                 168       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 82,045
Trainable params: 82,045
Non-trainable params: 0
_________________________________________________________________


# Train the Recurrent Neural Network

In [406]:
%%time
model.fit(X_train_pad, Y_train,
          validation_split=0.05, epochs=3, batch_size=64)

Epoch 1/3
594/594 [==============================] - 344s 580ms/step - loss: 0.6289 - accuracy: 0.6376 - val_loss: 0.5995 - val_accuracy: 0.6860
Epoch 2/3
594/594 [==============================] - 326s 550ms/step - loss: 0.5924 - accuracy: 0.6829 - val_loss: 0.5870 - val_accuracy: 0.6885
Epoch 3/3
594/594 [==============================] - 320s 539ms/step - loss: 0.5834 - accuracy: 0.6872 - val_loss: 0.5855 - val_accuracy: 0.6790
Wall time: 16min 53s


In [407]:
X_test_pad= np.asarray(X_test_pad).astype(np.float32)

In [408]:
#performance of the tests

#%%time
result = model.evaluate(X_test_pad, Y_test)

157/157 [==============================] - 13s 80ms/step - loss: 0.5824 - accuracy: 0.6926


In [409]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 69.26%
